# Demo of parsing Markdown nicely

## Imports

In [1]:
# Loading and Saving files & others
import pathlib
import sys
import numpy as np

# Parse Markdown
from markdown_it import MarkdownIt # pip install markdown-it-py 
from mdformat.renderer import MDRenderer # pip install mdformat

# Dealing with YAML
import yaml

# deal with multi-line strings in YAML Dump
## Code copied from here: https://stackoverflow.com/a/33300001/2217577

def str_presenter(dumper, data):
    if len(data.splitlines()) > 1:  # check for multiline string
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

## Read files

In [2]:
# Read the markdown file 

path = 'source/q1_multiple-choice/q1_multiple-choice.md'
mdtext = pathlib.Path(path).read_text()

# Deal with YAML header
header_text = mdtext.rsplit('---\n')[1]
header = yaml.safe_load('---\n' + header_text)

# Deal with Markdown Body
body = mdtext.rsplit('---\n')[2]

## Parse markdown body

In [3]:
# Set up the markdown parser
# to be honest, not fully sure what's going on here, see this issue: https://github.com/executablebooks/markdown-it-py/issues/164

mdit = MarkdownIt()
env = {}

# Set up tokens by parsing the md file
tokens = mdit.parse(body, env)

In [4]:
blocks = {}

block_count = 0

num_titles = 0

for x,t in enumerate(tokens):
     
    if t.tag == 'h1' and t.nesting == 1: # title
        blocks['title'] = [x,]
        num_titles += 1
        
    elif t.tag == 'h2' and t.nesting == 1:
        block_count += 1
        
        if block_count == 1:
            blocks['block{0}'.format(block_count)] = [x,]
        else:
            blocks['block{0}'.format(block_count-1)].append(x)
            blocks['block{0}'.format(block_count)] = [x,]

    #print(t,'\n')
    
# Add -1 to the end of the last block
blocks['block{0}'.format(block_count)].append(len(tokens))

# Assert statements (turn into tests!)
assert num_titles == 1, "I see {0} Level 1 Headers (#) in this file, there should only be one!".format(num_titles)
assert block_count > 1, "I see {0} Level 2 Headers (##) in this file, there should be at least 1".format(block_count -1)

# Add the end of the title block; # small hack
blocks['title'].append(blocks['block1'][0])

In [5]:
blocks

{'title': [0, 3],
 'block1': [3, 44],
 'block2': [44, 85],
 'block3': [85, 91],
 'block4': [91, 97]}

In [6]:
## Process the blocks into markdown

body_parts = {}

part_counter = 0

for k,v in blocks.items():

    rendered_part = MDRenderer().render(tokens[v[0]:v[1]], mdit.options, env)
    
    if k == 'title':
        body_parts['title'] = rendered_part
    
    elif 'Rubric' in rendered_part:
        body_parts['Rubric'] = rendered_part

    elif 'Solution' in rendered_part:
        body_parts['Solution'] = rendered_part

    elif 'Comments' in rendered_part:
        body_parts['Comments'] = rendered_part
    
    else:
        part_counter +=1
        body_parts['part{0}'.format(part_counter)] = rendered_part

In [7]:
body_parts.keys()

dict_keys(['title', 'part1', 'part2', 'Rubric', 'Solution'])

In [8]:
header['part1']

{'type': 'multiple-choice',
 'choices': 'ans1, ans2, ans3, ans4',
 'answer': 'ans1',
 'units': 'm/s',
 'pl-options': {'allow-blank': True}}

In [9]:
print(body_parts['part1'])

## Part A

{{ vars.name }} is traveling on {{ vars.vehicle }} at {{ params.v }} {{ vars.units }}.
How far does {{ vars.name }} travel in {{ params.t }} seconds, assuming they continue at the same velocity?

### Answer Section

- {{ params.ans1}} {{ vars.units}}
- {{ params.ans2}} {{ vars.units}}
- {{ params.ans3}} {{ vars.units}}
- {{ params.ans4}} {{ vars.units}}
- {{ params.ans5}} {{ vars.units}}
- {{ params.ans6}} {{ vars.units}}



## Helper functions

In [23]:
def rounded(num, digits_after_decimal = 2):
    
    """
    Rounding as expected by normal sensible people.
    
    This needs to be heavily tested!!
    
    WARNING: This does not do sig figs yet!
    
    """

    # Solution copied from: https://stackoverflow.com/a/53329223

    from decimal import Decimal, getcontext, ROUND_HALF_UP

    round_context = getcontext()
    round_context.rounding = ROUND_HALF_UP
        
    tmp = Decimal(num).quantize(Decimal('1.'+'0'*digits_after_decimal))
    
    return str(tmp)

def dict_to_md(md_dict, remove_keys = [None,]):
    
    md_string = ""
    
    for k,v in md_dict.items():
        if k in remove_keys:
            continue
        else:
            md_string += md_dict[k]
            
    return md_string

## Process Files

In [21]:
def process_files(header, body_parts, source_file, 
                  instructor=True, prairielearn=False, public=True, 
                  qti=False, webwork=False):
    """
    Inputs:
    
    - path to source file
    - output types
    
    Outputs:
    
    - processed files to each of the output types
    """
        
    # populate the data dictionary after running generate function
    data={'params':{},
          'vars': {}}

    ## TODO: Is there a better way to do this?
    exec(header['server'])

    # This is a Completely incomrephensible dictionary comprehension
    # All this does is take every number in the data['params'] dictionary, and turns it into a sensibly rounded number, 
    # It also leaves strings as-is (like correct-answer)

    data['params'] = {key : rounded(data['params'][key], digits_after_decimal=data["vars"]["digits_after_decimal"]) if type(data['params'][key]) != str else data['params'][key] for key in data['params']}

    if instructor:
        
        instructor_header = header.copy()
                
        # Update the YAML header to add substitutions 
        instructor_header.update({'substitutions':data})
        
        # Update the YAML header to add substitutions, unsort it, and process for file
        instructor_header = yaml.dump(instructor_header,sort_keys=False,default_flow_style=False)

        # Write the YAML to a file
        path = pathlib.Path(source_file.replace('source','output/instructor'))
        path.parent.mkdir(parents=True, exist_ok=True)
        path.write_text('---\n' + instructor_header + '---\n' + dict_to_md(body_parts))
        
    if public:
        
        sanitized_header = header.copy()
        
        # Remove python solution from the public view
        sanitized_header.pop('server',None)

        # Remove the correct answer from the dictionary
        data["params"]["correct_answer"] = 'nan'

        # Update the YAML header to add substitutions 
        sanitized_header.update({'substitutions':data})

        # Update the YAML header to add substitutions, unsort it, and process for file
        sanitized_header = yaml.dump(sanitized_header,sort_keys=False,default_flow_style=False)

        # Write the YAML to a file
        path = pathlib.Path(source_file.replace('source','output/public'))
        path.parent.mkdir(parents=True, exist_ok=True)
        path.write_text('---\n' + sanitized_header + '---\n' + dict_to_md(body_parts,remove_keys=['Rubric','Solution','Comments']))

    if webwork:
        raise NotImplementedError
        
    if prairielearn:
        raise NotImplementedError
    

In [22]:
process_files(header,body_parts, source_file = 'source/q1_multiple-choice/q1_multiple-choice.md')